In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# From Tables to Neural Networks: The DQN Architecture -- Vizuara

## 1. Why Does This Matter?

In our previous exploration of Q-learning, we built agents that maintained a table of Q-values -- one entry for every state-action pair. This worked beautifully for small grid worlds with a handful of states.

But what happens when we want an agent to play Atari games directly from raw pixel input?

A single Atari frame is 210 x 160 pixels with 128 possible color values per pixel. The number of possible states is $128^{33{,}600}$ -- a number so incomprehensibly large that no table could ever store it. Even a simplified 10x10 grid with 4 colors gives $4^{100} \approx 10^{60}$ states, more than the atoms in the observable universe.

This is the problem that DeepMind solved in 2013 with Deep Q-Networks (DQN). Instead of memorizing Q-values, they trained a convolutional neural network to **approximate** them. The same architecture, the same hyperparameters, applied to 49 different Atari games -- and it achieved superhuman performance on 29 of them.

In this notebook, we will build the DQN neural network from scratch. By the end, you will have a working CNN that takes raw pixel frames as input and outputs Q-values for every possible action.

---

## 2. Building Intuition

### Why tables fail

Let us start with something concrete. We will build a small Q-table and see exactly where it breaks down.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

# A tiny 3x3 grid world -- Q-table works fine
n_states_small = 9
n_actions = 4  # up, down, left, right

# Initialize a Q-table
q_table = np.zeros((n_states_small, n_actions))
print(f"Q-table for 3x3 grid: {q_table.shape}")
print(f"Total entries to store: {q_table.size}")
print(f"Memory: {q_table.nbytes} bytes\n")

# Now try Atari-scale
# Even a tiny grayscale 84x84 image with 256 values per pixel
pixels = 84 * 84  # 7056 pixels
values_per_pixel = 256
print(f"For an 84x84 grayscale image:")
print(f"  Pixels: {pixels}")
print(f"  Possible states: 256^{pixels}")
print(f"  That is approximately 10^{int(pixels * np.log10(256))}")
print(f"  Atoms in the universe: ~10^80")
print(f"  Ratio: 10^{int(pixels * np.log10(256)) - 80} times more states than atoms!")

### The function approximation idea

Instead of memorizing every face you have ever seen, your brain learns general features -- the shape of eyes, the curve of a nose -- and combines them to recognize new faces instantly. Function approximation does the same thing for Q-values.

In [ ]:
# Instead of a lookup table, we want a FUNCTION:
# Q(state, action; theta) -> value
#
# Let us see this with a simple example.
# True Q-values for a tiny problem:
true_q = {
    'state_A': [2.0, 5.0],  # action 0 -> 2.0, action 1 -> 5.0
    'state_B': [3.0, 1.0],
    'state_C': [4.0, 4.5],
}

# A table stores all 6 values explicitly.
# A neural network LEARNS the patterns and can generalize.

# Simple demo: approximate with a linear function
# State features: [x_position, y_position]
states = torch.tensor([
    [0.0, 0.0],  # state_A
    [1.0, 0.0],  # state_B
    [0.5, 1.0],  # state_C
])
targets = torch.tensor([
    [2.0, 5.0],
    [3.0, 1.0],
    [4.0, 4.5],
])

# A simple linear approximator
linear_q = nn.Linear(2, 2)
optimizer = torch.optim.Adam(linear_q.parameters(), lr=0.01)

losses = []
for step in range(500):
    pred = linear_q(states)
    loss = F.mse_loss(pred, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

plt.figure(figsize=(8, 3))
plt.plot(losses)
plt.xlabel('Training Step')
plt.ylabel('MSE Loss')
plt.title('Linear function learning to approximate Q-values')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTrue Q-values:        {targets.numpy()}")
print(f"Approximated Q-values: {linear_q(states).detach().numpy().round(2)}")
print(f"\nThe function learned to approximate the Q-values from features!")

### Why CNNs for pixel input?

When states are raw images, we need a function that understands spatial patterns -- edges, shapes, objects. Convolutional neural networks are purpose-built for this.

In [ ]:
# Visualize what convolution does to a game-like image
from matplotlib.patches import Rectangle

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Create a simple "game screen" with a paddle and ball
game_screen = np.zeros((84, 84))
# Paddle at bottom
game_screen[75:80, 30:50] = 1.0
# Ball
game_screen[40:44, 42:46] = 1.0
# Bricks at top
for row in range(3):
    for col in range(8):
        game_screen[5+row*5:5+row*5+3, 5+col*10:5+col*10+8] = 0.7

axes[0].imshow(game_screen, cmap='gray')
axes[0].set_title('Input: 84x84 game frame')
axes[0].axis('off')

# Apply a simple edge-detection kernel (like Conv1 might learn)
from scipy.ndimage import convolve
edge_kernel = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
edges = convolve(game_screen, edge_kernel)
axes[1].imshow(np.abs(edges), cmap='hot')
axes[1].set_title('After edge detection (Conv layer)')
axes[1].axis('off')

# Show that the CNN extracts meaningful features
axes[2].imshow(game_screen, cmap='gray')
axes[2].add_patch(Rectangle((30, 75), 20, 5, linewidth=2, edgecolor='cyan', facecolor='none'))
axes[2].add_patch(Rectangle((42, 40), 4, 4, linewidth=2, edgecolor='red', facecolor='none'))
axes[2].set_title('CNN learns to detect: paddle, ball, bricks')
axes[2].axis('off')

plt.suptitle('Why CNNs? They extract spatial features from raw pixels', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 3. The Mathematics

The core idea is to replace the Q-table with a parameterized function:

$$Q(s, a; \theta) \approx Q^*(s, a)$$

where $\theta$ represents the neural network weights.

For a convolutional layer, the output spatial dimension is:

$$o = \frac{i - k + 2p}{s} + 1$$

where $i$ is input size, $k$ is kernel size, $p$ is padding, and $s$ is stride.

Let us trace through the DQN architecture dimensions step by step.

In [ ]:
# Let us compute the DQN dimensions mathematically
def conv_output_size(input_size, kernel_size, stride, padding=0):
    """Calculate output size of a conv layer."""
    return (input_size - kernel_size + 2 * padding) // stride + 1

# DQN architecture dimensions
input_size = 84

# Conv1: 32 filters, 8x8 kernel, stride 4
after_conv1 = conv_output_size(input_size, kernel_size=8, stride=4)
print(f"Input: {input_size}x{input_size}")
print(f"After Conv1 (8x8, stride 4): {after_conv1}x{after_conv1} with 32 channels")

# Conv2: 64 filters, 4x4 kernel, stride 2
after_conv2 = conv_output_size(after_conv1, kernel_size=4, stride=2)
print(f"After Conv2 (4x4, stride 2): {after_conv2}x{after_conv2} with 64 channels")

# Conv3: 64 filters, 3x3 kernel, stride 1
after_conv3 = conv_output_size(after_conv2, kernel_size=3, stride=1)
print(f"After Conv3 (3x3, stride 1): {after_conv3}x{after_conv3} with 64 channels")

# Flatten
flat_size = 64 * after_conv3 * after_conv3
print(f"\nFlattened size: 64 x {after_conv3} x {after_conv3} = {flat_size}")
print(f"FC layer: {flat_size} -> 512 -> n_actions")

# Visualize the dimension reduction
sizes = [84, after_conv1, after_conv2, after_conv3]
channels = [4, 32, 64, 64]
total_values = [s*s*c for s, c in zip(sizes, channels)]

fig, ax = plt.subplots(figsize=(10, 4))
labels = ['Input\n4x84x84', 'Conv1\n32x20x20', 'Conv2\n64x9x9', 'Conv3\n64x7x7']
colors = ['#3498db', '#2ecc71', '#e67e22', '#e74c3c']
bars = ax.bar(labels, total_values, color=colors, edgecolor='white', linewidth=2)
for bar, val in zip(bars, total_values):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 200,
            f'{val:,}', ha='center', va='bottom', fontweight='bold')
ax.set_ylabel('Total Values (height x width x channels)')
ax.set_title('DQN progressively compresses spatial information')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---

## 4. Let's Build It -- Component by Component

### Step 1: The DQN network

In [ ]:
class DQN(nn.Module):
    """
    Deep Q-Network as described in Mnih et al. (2013/2015).

    Input:  (batch, 4, 84, 84) -- 4 stacked grayscale frames
    Output: (batch, n_actions)  -- Q-value for each action
    """
    def __init__(self, n_actions):
        super(DQN, self).__init__()

        # Convolutional backbone -- extracts spatial features
        self.conv = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),  # -> 32x20x20
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), # -> 64x9x9
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), # -> 64x7x7
            nn.ReLU()
        )

        # Fully connected head -- maps features to Q-values
        self.fc = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        # x shape: (batch, 4, 84, 84), pixel values in [0, 1]
        conv_out = self.conv(x)
        flat = conv_out.view(conv_out.size(0), -1)  # Flatten to (batch, 3136)
        return self.fc(flat)

# Create the network
n_actions = 4  # Typical Atari game (e.g., Breakout: noop, fire, left, right)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dqn = DQN(n_actions).to(device)

# Count parameters
total_params = sum(p.numel() for p in dqn.parameters())
print(f"DQN Network Summary:")
print(f"  Device: {device}")
print(f"  Total parameters: {total_params:,}")
print(f"  That is {total_params * 4 / 1024 / 1024:.1f} MB of weights (float32)\n")

# Print layer details
for name, param in dqn.named_parameters():
    print(f"  {name:30s} {str(list(param.shape)):20s} ({param.numel():>8,} params)")

### Step 2: Verify with a forward pass

In [ ]:
# Create a fake batch of 4-frame stacks
batch_size = 8
fake_input = torch.randn(batch_size, 4, 84, 84).to(device)

# Forward pass
with torch.no_grad():
    q_values = dqn(fake_input)

print(f"Input shape:  {fake_input.shape}")
print(f"Output shape: {q_values.shape}")
print(f"\nQ-values for first sample:")
for i, q in enumerate(q_values[0]):
    print(f"  Action {i}: Q = {q.item():.4f}")

# The action we would select (greedy policy)
best_action = q_values[0].argmax().item()
print(f"\nGreedy action: {best_action} (highest Q-value)")

### Step 3: Trace through the conv layers

In [ ]:
# Let us manually trace through each layer to verify our math
x = torch.randn(1, 4, 84, 84).to(device)

print("Tracing through DQN layers:")
print(f"  Input:         {x.shape}")

# Conv1
x1 = dqn.conv[0](x)  # Conv2d
print(f"  After Conv1:   {x1.shape}  (expected: [1, 32, 20, 20])")
x1 = dqn.conv[1](x1)  # ReLU

# Conv2
x2 = dqn.conv[2](x1)  # Conv2d
print(f"  After Conv2:   {x2.shape}  (expected: [1, 64, 9, 9])")
x2 = dqn.conv[3](x2)  # ReLU

# Conv3
x3 = dqn.conv[4](x2)  # Conv2d
print(f"  After Conv3:   {x3.shape}  (expected: [1, 64, 7, 7])")
x3 = dqn.conv[5](x3)  # ReLU

# Flatten
flat = x3.view(1, -1)
print(f"  After flatten: {flat.shape} (expected: [1, 3136])")

# FC
out = dqn.fc(flat)
print(f"  Output:        {out.shape}  (expected: [1, {n_actions}])")
print(f"\nAll dimensions match the DQN paper exactly.")

---

## 5. Your Turn

### TODO 1: Modify the architecture

The original DQN uses ReLU activations. What happens if you use a different activation function?

In [ ]:
# TODO: Create a modified DQN that uses LeakyReLU instead of ReLU
# Compare the output distributions of both networks on the same input

class DQN_LeakyReLU(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        # YOUR CODE HERE
        # Hint: Replace nn.ReLU() with nn.LeakyReLU(0.01)
        self.conv = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),
            nn.LeakyReLU(0.01),  # TODO: already filled in as example
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.LeakyReLU(0.01),
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512),
            nn.LeakyReLU(0.01),
            nn.Linear(512, n_actions),
        )

    def forward(self, x):
        # YOUR CODE HERE
        conv_out = self.conv(x)
        flat = conv_out.view(conv_out.size(0), -1)
        return self.fc(flat)

# After implementing, run this comparison:
dqn_leaky = DQN_LeakyReLU(n_actions).to(device)
test_input = torch.randn(100, 4, 84, 84).to(device)
with torch.no_grad():
    q_relu = dqn(test_input)
    q_leaky = dqn_leaky(test_input)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].hist(q_relu.cpu().numpy().flatten(), bins=50, alpha=0.7, color='blue')
axes[0].set_title('ReLU Q-value distribution')
axes[1].hist(q_leaky.cpu().numpy().flatten(), bins=50, alpha=0.7, color='green')
axes[1].set_title('LeakyReLU Q-value distribution')
plt.tight_layout()
plt.show()

### TODO 2: Compute the receptive field

Each output neuron "sees" a specific region of the input image. This is called the receptive field.

In [ ]:
# TODO: Calculate the receptive field of the final conv layer
#
# The receptive field formula for layer L is:
#   RF_L = RF_{L-1} + (kernel_size_L - 1) * stride_product_{1..L-1}
#
# Starting with RF_0 = 1:
#   Conv1: kernel=8, stride=4
#   Conv2: kernel=4, stride=2
#   Conv3: kernel=3, stride=1
#
# YOUR CODE HERE:
# rf_0 = 1
# rf_1 = rf_0 + (8 - 1) * 1      # = ?
# rf_2 = rf_1 + (4 - 1) * 4      # = ? (stride product so far: 4)
# rf_3 = rf_2 + (3 - 1) * (4*2)  # = ? (stride product so far: 4*2=8)
#
# print(f"Receptive field of Conv3 output: {rf_3}x{rf_3} pixels")
# print(f"Input image: 84x84 pixels")
# print(f"Each output neuron 'sees' {rf_3/84*100:.1f}% of the input")

---

## 6. Putting It All Together

Now let us build the complete frame preprocessing pipeline and connect it to our DQN.

In [ ]:
import cv2
from collections import deque

class AtariPreprocessor:
    """
    Preprocesses raw Atari frames for the DQN.

    Steps:
    1. Convert RGB to grayscale
    2. Resize to 84x84
    3. Normalize to [0, 1]
    4. Stack 4 consecutive frames
    """
    def __init__(self, frame_stack=4, resize_shape=(84, 84)):
        self.frame_stack = frame_stack
        self.resize_shape = resize_shape
        self.frames = deque(maxlen=frame_stack)

    def preprocess_frame(self, frame):
        """Convert a single raw frame to grayscale 84x84."""
        if len(frame.shape) == 3:
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        else:
            gray = frame
        resized = cv2.resize(gray, self.resize_shape)
        return resized.astype(np.float32) / 255.0

    def reset(self, frame):
        """Initialize the frame stack with copies of the first frame."""
        processed = self.preprocess_frame(frame)
        for _ in range(self.frame_stack):
            self.frames.append(processed)
        return torch.tensor(np.array(self.frames))

    def step(self, frame):
        """Add a new frame to the stack."""
        processed = self.preprocess_frame(frame)
        self.frames.append(processed)
        return torch.tensor(np.array(self.frames))

# Demo: preprocess a synthetic game frame
preprocessor = AtariPreprocessor()

# Create a fake RGB Atari frame (210x160x3)
fake_frame = np.random.randint(0, 256, (210, 160, 3), dtype=np.uint8)
# Add a "paddle" and "ball" for visual clarity
fake_frame[180:190, 60:100, :] = [255, 255, 255]  # paddle
fake_frame[100:106, 78:84, :] = [255, 0, 0]       # ball

# Process it
stacked = preprocessor.reset(fake_frame)
print(f"Raw frame:    {fake_frame.shape} (H x W x C, uint8)")
print(f"Preprocessed: {stacked.shape}   (C x H x W, float32)")

# Visualize the preprocessing pipeline
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(fake_frame)
axes[0].set_title(f'Raw RGB ({fake_frame.shape[0]}x{fake_frame.shape[1]}x3)')
axes[0].axis('off')

single_processed = preprocessor.preprocess_frame(fake_frame)
axes[1].imshow(single_processed, cmap='gray')
axes[1].set_title(f'Grayscale + Resized (84x84)')
axes[1].axis('off')

# Show the 4-frame stack as a grid
axes[2].imshow(stacked[0].numpy(), cmap='gray')
axes[2].set_title(f'Frame stack: {stacked.shape}')
axes[2].axis('off')

plt.suptitle('Atari Preprocessing Pipeline', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Connect the preprocessor to the DQN
stacked_input = stacked.unsqueeze(0).to(device)  # Add batch dimension

with torch.no_grad():
    q_values = dqn(stacked_input)

print(f"Input to DQN:  {stacked_input.shape}")
print(f"Q-values out:  {q_values.shape}")
print(f"\nQ-values for each action:")
action_names = ['NOOP', 'FIRE', 'LEFT', 'RIGHT']
for i, (name, q) in enumerate(zip(action_names, q_values[0])):
    marker = ' <-- BEST' if i == q_values[0].argmax().item() else ''
    print(f"  {name:6s}: Q = {q.item():+.4f}{marker}")

print(f"\nThe DQN takes 4 stacked 84x84 frames and outputs one Q-value per action.")
print(f"The agent selects the action with the highest Q-value.")

---

## 7. Training and Results

Let us verify our network can actually learn something meaningful. We will train the DQN on a simplified pattern recognition task to confirm the architecture works correctly.

In [ ]:
# Simplified test: can the DQN learn to distinguish different "game states"?
# We create synthetic frames with different patterns and train it to output
# the correct Q-values.

# Generate training data: 4 classes of game states
def generate_synthetic_state(class_id):
    """Generate a synthetic 4x84x84 game state."""
    frames = np.zeros((4, 84, 84), dtype=np.float32)
    for f in range(4):
        if class_id == 0:  # Ball top-left
            frames[f, 10:16, 10:16] = 1.0
            frames[f, 70:76, 30+f*3:50+f*3] = 1.0  # Moving paddle
        elif class_id == 1:  # Ball top-right
            frames[f, 10:16, 68:74] = 1.0
            frames[f, 70:76, 30+f*3:50+f*3] = 1.0
        elif class_id == 2:  # Ball bottom-left
            frames[f, 55:61, 10:16] = 1.0
            frames[f, 70:76, 30+f*3:50+f*3] = 1.0
        else:  # Ball bottom-right
            frames[f, 55:61, 68:74] = 1.0
            frames[f, 70:76, 30+f*3:50+f*3] = 1.0
    return torch.tensor(frames)

# Training
dqn_train = DQN(4).to(device)
optimizer = torch.optim.Adam(dqn_train.parameters(), lr=1e-3)

# Target Q-values: each class should prefer a different action
target_q = torch.tensor([
    [5.0, 0.0, 0.0, 0.0],  # Class 0 -> action 0
    [0.0, 5.0, 0.0, 0.0],  # Class 1 -> action 1
    [0.0, 0.0, 5.0, 0.0],  # Class 2 -> action 2
    [0.0, 0.0, 0.0, 5.0],  # Class 3 -> action 3
]).to(device)

losses = []
accuracies = []

for epoch in range(200):
    batch_states = []
    batch_targets = []

    for class_id in range(4):
        for _ in range(8):  # 8 samples per class
            state = generate_synthetic_state(class_id)
            batch_states.append(state)
            batch_targets.append(target_q[class_id])

    states = torch.stack(batch_states).to(device)
    targets = torch.stack(batch_targets)

    pred_q = dqn_train(states)
    loss = F.mse_loss(pred_q, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Check accuracy
    pred_actions = pred_q.argmax(dim=1)
    true_actions = targets.argmax(dim=1)
    accuracy = (pred_actions == true_actions).float().mean().item()

    losses.append(loss.item())
    accuracies.append(accuracy)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(losses, color='#e74c3c')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('MSE Loss')
ax1.set_title('Training Loss')
ax1.grid(True, alpha=0.3)

ax2.plot(accuracies, color='#2ecc71')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Action Selection Accuracy')
ax2.set_ylim(-0.05, 1.05)
ax2.grid(True, alpha=0.3)

plt.suptitle('DQN learns to map different visual patterns to correct actions', fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nFinal loss: {losses[-1]:.4f}")
print(f"Final accuracy: {accuracies[-1]:.1%}")
print(f"\nThe DQN architecture can learn visual pattern -> action mappings!")

---

## 8. Final Output

In [ ]:
# Final demonstration: the trained DQN correctly classifies game states
print("=" * 60)
print("FINAL VERIFICATION: DQN Architecture")
print("=" * 60)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for class_id in range(4):
    state = generate_synthetic_state(class_id).unsqueeze(0).to(device)
    with torch.no_grad():
        q_vals = dqn_train(state)

    axes[class_id].imshow(state[0, 0].cpu().numpy(), cmap='gray')
    axes[class_id].set_title(f'State class {class_id}')
    axes[class_id].axis('off')

    action = q_vals.argmax().item()
    q_str = ', '.join([f'{q:.1f}' for q in q_vals[0].cpu().numpy()])
    axes[class_id].set_xlabel(f'Q: [{q_str}]\nAction: {action}', fontsize=9)

plt.suptitle('DQN correctly maps visual states to actions', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\nNetwork architecture: 4x84x84 -> Conv(32) -> Conv(64) -> Conv(64) -> FC(512) -> {n_actions} actions")
print(f"Total parameters: {sum(p.numel() for p in dqn_train.parameters()):,}")
print(f"The same architecture is used for ALL 49 Atari games in the DQN paper.")

---

## 9. Reflection and Next Steps

**What we built:**
- A complete DQN convolutional neural network that maps raw pixel frames to Q-values
- An Atari preprocessing pipeline (RGB -> grayscale -> resize -> frame stack)
- Verified the architecture can learn visual pattern-to-action mappings

**Key dimensions to remember:**
- Input: 4 x 84 x 84 (4 stacked grayscale frames)
- Conv1: 32 filters, 8x8, stride 4 -> 32 x 20 x 20
- Conv2: 64 filters, 4x4, stride 2 -> 64 x 9 x 9
- Conv3: 64 filters, 3x3, stride 1 -> 64 x 7 x 7
- Flatten: 3136 -> FC: 512 -> Output: n_actions

**Think about:**
1. Why did DeepMind choose these specific kernel sizes and strides? What happens if you use smaller kernels with stride 1 everywhere?
2. The network has no pooling layers. Why might that be important for game playing?
3. Why 4 stacked frames instead of 2 or 8?

**Next notebook:** We will tackle the two innovations that made training this network stable -- experience replay and target networks. Without them, the DQN architecture we just built would fail to learn anything useful.